In [16]:
import sys
import h5py
import glob
import numpy as np 
from numpy import random
import matplotlib.pyplot as plt
from scipy import signal
import scipy as sp
from scipy.signal import butter
from scipy.optimize import curve_fit
from lmfit.models import GaussianModel
import time
import random

In [267]:
# reading h5 file
f = h5py.File("20190518_173944_projection-Copy1.h5")
list(f.keys())
shots = f.get("n-shots")
photE = f.get("x-axis")
intense = f.get("y-axis")
spectrum = range(0,len(intense[1,:]-1)) # for parameter redefinition
x = np.asarray(spectrum)

In [571]:
# Peak finding parameters
heightcut = 10 # percentage height of max a peak must be
prominence = 5 # percentage prominence required for peak

In [572]:
# Lowpass/backloop conditions
deg = 5 # lowpass function degree. Chosen by inspection.
threshold = 10 # percentage overlap threshold for spectra to be sent through slow fit
backloop_condition_slow = 5
backloop_condition = 8 # percentage height difference between max of raw and max of lp function (maybe determine in terms of noise)

In [ ]:
start = time.time()
print("Optimising lowpass cutoff for fast fit")
cutoff = Fast_Backloop().cutoff()
spectra_for_slow_fit = []
for i in range(len(intense[:,1])):
    fn = Fast_Fit(intense[i,:],deg,cutoff)
    if fn.U() == 1:
        spectra_for_slow_fit = np.append(spectra_for_slow_fit, i) 
    
    if i == 0:
        lpfns = fn.lpfn
        gaussians = fn.gauss()
        number_of_peaks = fn.nofpeaks()
        avgsigma = fn.avg_sigmas
    else:
        lpfns = np.vstack((lpfns, fn.lpfn))
        gaussians = np.vstack((gaussians, fn.gauss()))
        number_of_peaks = np.vstack((number_of_peaks, fn.nofpeaks()))
        avgsigma = np.vstack((avgsigma, fn.avg_sigmas))

intermediate = time.time()
print("Time taken for fast fit:", (intermediate - start)/60, "minutes")
print("Number of spectra to be analysed by slow fit: ", len(spectra_for_slow_fit))
print("Beginning slow fit")

for k in spectra_for_slow_fit:
    k = int(k)
    fn = Slow_Fit(intense[k,:])
    lpfns[k] = fn.lpfn
    gaussians[k] = fn.gauss()
    number_of_peaks[k] = fn.nofpeaks()
    avgsigma[k] = fn.avg_sigmas
print("Data replaced")
end = time.time()
print("Time taken for slow fit:", (end - intermediate)/60, "minutes")
print("Time taken overall:", (end - start)/60, "minutes")

Optimising lowpass cutoff for fast fit
Trialled for spectrum 55
Trialled for spectrum 80
Trialled for spectrum 75
Trialled for spectrum 31
Average cutoff for dataset: 0.0073


In [ ]:
# replace slow-fitted datasets into previous datasets

In [594]:
class Slow_Fit:
    def __init__(self, lowpassdata):
        self.lowpassdata = lowpassdata
        
        # fitting lowpass, including backloop for better cutoff.
        lpcutoff = 0.0001
        while True:
            # fitting lowpass function with given cutoff
            b, a = signal.butter(deg, lpcutoff, 'low')
            self.spec = signal.filtfilt(b, a, self.lowpassdata)
                
            # Visual module for checking
            """plt.plot(self.lpfn, label='lowpass')
            plt.plot(self.lowpassdata, label='raw data')
            plt.legend()
            plt.show()"""

            # condition for good fitting based on vertical distance between maxima of raw and lowpass dataset
            # if not fitted within backloop_condition, increment until good fit
            self.height_difference = abs(max(self.lowpassdata) - max(self.spec))
            if self.height_difference > backloop_condition_slow * max(self.lowpassdata)/100:
                lpcutoff = lpcutoff + 0.001
            else:
                break
        
        # shift spectrum
        self.lpfn = self.spec - min(self.spec)
        
        # calculate noise
        self.noise = np.array([self.lowpassdata-self.spec]).T
        
        # Extracting peaks ([x-axis, y-axis]) - indexed to neutral
        self.peaks = Peakfinder(self.lpfn).peaks
        
        # Apply further constraints to peaks
        self.filteredpeaks = Filter_peaks(self.peaks).filtered_peaks()
                
        # number of peaks, after filtering.
        self.n = len(self.filteredpeaks)
        
        fn = Slice(self.lpfn,self.filteredpeaks, self.n)
        
        # Slice into individual peak functions, outside peak spectrum = 0. 
        self.slices = fn.slices()
        
        # Approximate slices with Gaussians
        
        self.fn1 = Gauss(self.slices, self.n)
        self.Gaussian = self.fn1.Added_Gaussian()  
        self.IndivGauss = self.fn1.IndivGaussians()
        
        slicepos = fn.SlicingPoints()
        
        self.fn2 = MaxMin(self.IndivGauss,self.n, slicepos, self.lpfn)
        
        plt.plot(photE, self.Gaussian, label="Gaussian approximation")
        for i in range(self.IndivGauss.shape[0]):
            plt.plot(photE, self.IndivGauss[i],'--',markersize = 0.1, label="indiv gaussian")
        plt.plot(photE, self.lpfn, label = "shifted lowpass function")
        
        plt.plot(self.fn2.GetMax_x(),self.fn2.GetMax_y(), 'go')
        if self.n != 1:
            plt.plot(self.fn2.GetMin_x(),self.fn2.GetMin_y(), 'ro')
        plt.legend()
        plt.show()
        
        minima = fn.SlicingPoints()
        self.u = 0
        for i in range(len(minima)):
            index = int(minima[i])
            if (abs(self.Gaussian[index]-self.lpfn[index])*100/max(self.lpfn)) > threshold and self.u == 0:
                self.u = self.u + 1
                
    def U(self):
        return self.u
    def lpfn(self):
        return self.lpfn
    def gauss(self):
        return self.Gaussian
    def indiv(self):
        return self.IndivGauss
    def nofpeaks(self):
        return self.n
    def avg_sigmas(self):
        return np.average(self.fn1.sigmas()) 
        

In [ ]:
# all analysis can go here

In [585]:
class MaxMin:
    def __init__(self, data, nofpeaks, slicepos, lpfn):
        self.n = nofpeaks
        self.data = data
        self.slicepos = slicepos
        self.lpfn = lpfn
        
        if self.n == 1:
            self.ymax = max(self.data)
            self.xmax = np.where(self.data == self.ymax)
            
        else:
            self.ymax = []
            self.xmax = []
            for i in range(self.n):
                ymax = max(self.data[i])
                self.ymax = np.append(self.ymax, ymax)
                
                xmax = np.where(self.data[i] == ymax)
                ind1 = int(xmax[0])
                xmax = photE[ind1]
                self.xmax = np.append(self.xmax, xmax)
            
            self.ymin = []
            self.xmin = []
            for j in range(len(slicepos)):
                ind2 = int(self.slicepos[j])
                ymin = self.lpfn[ind2]
                self.ymin = np.append(self.ymin, ymin)

                xmin = photE[self.slicepos[j]]
                self.xmin = np.append(self.xmin, xmin)
    
    def GetMax_x(self):
        return self.xmax
    def GetMax_y(self):
        return self.ymax
    def GetMin_x(self):
        return self.xmin
    def GetMin_y(self):
        return self.ymin            

In [488]:
class Fast_Fit:
    def __init__(self, lowpassdata, deg, cutoff):
        self.cutoff = float(cutoff)
        # lowpass function itself
        self.lowpassdata = lowpassdata
        b, a = signal.butter(deg, self.cutoff, 'low')
        self.spec = signal.filtfilt(b, a, self.lowpassdata)
        
        # shift spectrum
        self.lpfn = self.spec - min(self.spec)
        
        # calculate noise
        self.noise = np.array([self.lowpassdata-self.spec]).T
        
        """
        # visual module
        print("plotting lowpass data vs raw data, noise etc")
        plt.plot(photE,self.lowpassdata,label="raw data")
        plt.plot(photE,self.lpfn,label="shifted spectrum")
        plt.plot(photE,self.spec,label="lowpass")
        plt.plot(photE,self.noise,label="noise")
        plt.legend()
        plt.show()
        """
        
        # Extracting peaks ([x-axis, y-axis]) - indexed to neutral
        self.peaks = Peakfinder(self.lpfn).peaks
        
        # Apply further constraints to peaks
        self.filteredpeaks = Filter_peaks(self.peaks).filtered_peaks()
                
        # number of peaks, after filtering.
        self.n = len(self.filteredpeaks)
        
        fn = Slice(self.lpfn,self.filteredpeaks, self.n)
        
        # Slice into individual peak functions, outside peak spectrum = 0. 
        self.slices = fn.slices()
        
        # Approximate slices with Gaussians
        
        self.fn1 = Gauss(self.slices, self.n)
        self.Gaussian = self.fn1.Added_Gaussian()  
        self.IndivGauss = self.fn1.IndivGaussians()
        
        """
        # Visual module: comparing Gaussian to lowpass function - including individual gaussians
        plt.plot(self.Gaussian, label = "Added Gaussian")
        plt.plot(self.lpfn, label = "Shifted lowpass")
        for i in range(self.IndivGauss.shape[0]):
            plt.plot(self.IndivGauss[i],'--', markersize = '0.1', label = "indiv gaussian")
        plt.legend()
        plt.show()
        #"""
        
        minima = fn.SlicingPoints()
        self.u = 0
        for i in range(len(minima)):
            index = int(minima[i])
            if (abs(self.Gaussian[index]-self.lpfn[index])*100/max(self.lpfn)) > threshold and self.u == 0:
                self.u = self.u + 1
                
    def U(self):
        return self.u
    def lpfn(self):
        return self.lpfn
    def gauss(self):
        return self.Gaussian
    def indiv(self):
        return self.IndivGauss
    def nofpeaks(self):
        return self.n
    def avg_sigmas(self):
        return np.average(self.fn1.sigmas()) 

In [474]:
class Gauss:
    def __init__(self, data, n):
        self.data = data
        self.sigmas = []
        self.n = n
        
        if self.n == 1:
            peak = self.data
            mod = GaussianModel()
            pars = mod.guess(peak, x = x)
            out = mod.fit(peak, pars, x= x)
            gaussResult = out.best_fit
            self.gaussResults = gaussResult
            self.sigmas.append(out.params['sigma'].value)
        
        else:
            for i in range(len(self.data)):
                peak = self.data[i,:]
                mod = GaussianModel()
                pars = mod.guess(peak, x = x)
                out = mod.fit(peak, pars, x= x)
                gaussResult = out.best_fit

                # stacking Gaussians
                if i == 0:
                    self.gaussResults = gaussResult
                else:
                    self.gaussResults = np.vstack((self.gaussResults,gaussResult))

                self.sigmas.append(out.params['sigma'].value)
        
    def IndivGaussians(self):
        return self.gaussResults
        
    def sigmas(self):
        return self.sigmas
        
    def Added_Gaussian(self):
        if self.n == 1:
            GaussAdd = self.gaussResults
            
        else:
            GaussAdd = []
            for o in range(len(self.gaussResults[1])):
                GaussAdd = np.append(GaussAdd, 0)
                for t in range(len(self.gaussResults[:,1])):
                    GaussAdd[o] = GaussAdd[o] + self.gaussResults[t,o]
                    
        return GaussAdd

In [520]:
class Slice:
    def __init__(self, data, peaks, nofpeaks):
        self.peaks = peaks
        self.data = data
        self.xpeaks = peaks 
        self.minima = []
        self.nofpeaks = nofpeaks
        for j in range(self.nofpeaks-1):
            # Compute x-indices of all minima by which we slice
            peak1 = self.xpeaks[j]
            peak2 = self.xpeaks[j+1]
            Min_y = min(data[peak1:peak2])
            Min_x = np.asarray(np.where(self.data == Min_y))
            self.minima = np.append(self.minima, Min_x)      
    
        # To make following loop easier, include zeroth and last index of spectrum
        zero = np.insert(self.minima, 0, 0)
        self.minima_indices = np.append(zero, len(self.data))
        
        # Actually slicing, filling zeros where out of slice bounds
        for k in range(self.nofpeaks):
            if k == 0:
                index0 = self.minima_indices[k]
                index1 = self.xpeaks[k]/2
                index2 = self.minima_indices[k+1]
                single_slice = self.data[int(index1):int(index2)]
                for n in range(int(index0),int(index1)):
                    single_slice = np.insert(single_slice, 0, 0)
                for n in range(int(index2), len(self.data)):
                    single_slice = np.append(single_slice, 0)
                self.Slices = single_slice
                
            elif k == len(self.peaks)-1:
                index1 = self.minima_indices[k]
                index2 = self.xpeaks[k] + (len(self.data)-self.xpeaks[k])/2
                index0 = self.minima_indices[k+1]
                single_slice = self.data[int(index1):int(index2)]  
                for n in range(0,int(index1)):
                    single_slice = np.insert(single_slice, 0, 0)   
                for n in range(int(index2), int(index0)):
                    single_slice = np.append(single_slice, 0)  
                self.Slices = np.vstack((self.Slices,single_slice))
                
            else:
                single_slice = self.data[int(self.minima_indices[k]):int(self.minima_indices[k+1])]
                for n in range(0,int(self.minima_indices[k])):
                    single_slice = np.insert(single_slice, 0, 0)
                for n in range(int(self.minima_indices[k+1]), len(self.data)):
                    single_slice = np.append(single_slice, 0)
                self.Slices = np.vstack((self.Slices,single_slice))
            """
            # Visual module
            print("plotting single slices")
            plt.plot(single_slice)
            plt.show()
            """
        
    def SlicingPoints(self):
        return self.minima
    def slices(self):
        return self.Slices

In [313]:
# helps fast_fit return peaks
class Peakfinder:
    def __init__(self, data):
        self.data = data
        self.peaks = sp.signal.find_peaks(self.data, height = heightcut, prominence = prominence*max(self.data)/100)
        self.peaks = self.peaks[0]
        
        self.xpeaks = np.asarray(self.peaks)
        self.ypeaks = np.asarray(self.data[self.xpeaks])
        self.allpeaks = np.array([self.xpeaks,self.ypeaks])
        
        
    def peaks(self):
    # returns dataset with neutral index x and intensity y  
        """
        # Visual module
        print("Plotting peaks found. Neutrally indexed.")
        plt.plot(self.data, label="data fed to peak finder")
        plt.plot(self.xpeaks,self.ypeaks,label="peaks",'go')
        plt.legend()
        plt.show()
        """
        return self.allpeaks

In [314]:
# Can put further constraints in this class
class Filter_peaks:
    def __init__(self,data):
        self.data = data
        return None
    def filtered_peaks(self):
        return self.data

In [315]:
# Trialling lowpass function for random spectra to find optimal cutoff
# Indexed neutrally (i.e. not by energy)

# Potential upgrades: fit also "from above" -> i.e. take into account overfitting. Eg by max threshold of peaks
# Could also incorporate backloop into slowfit!

class Fast_Backloop:
    def __init__(self):
        cutoffs = []
        
        # finding appropriate cutoff for 4 spectra within dataset
        for i in range(4):

            p = random.randrange(0, len(intense[:,1]), 1) # finding random spectrum to use
            
            print ("Trialled for spectrum", p)
            
            self.lowpassdata = intense[p,:]
            lpcutoff = 0.001                         # this constant seems low enough - reconsider if not applicable to all datasets
            n = 0

            while True:
                # fitting lowpass function with given cutoff
                b, a = signal.butter(deg, lpcutoff, 'low')
                self.lpfn = signal.filtfilt(b, a, self.lowpassdata)
                
                # Visual module for checking
                """plt.plot(self.lpfn, label='lowpass')
                plt.plot(self.lowpassdata, label='raw data')
                plt.legend()
                plt.show()"""

                # condition for good fitting based on vertical distance between maxima of raw and lowpass dataset
                # if not fitted within backloop_condition, increment until good fit
                self.height_difference = abs(max(self.lowpassdata) - max(self.lpfn))
                if self.height_difference > backloop_condition * max(self.lowpassdata)/100:
                    lpcutoff = lpcutoff + 0.005
                else:
                    break

            cutoffs = np.append(cutoffs, lpcutoff)
        
        # Finding average of cutoffs to 4 decimal points
        self.avg_cutoff = "%.4f" % np.average(cutoffs)
        print("Average cutoff for dataset:", self.avg_cutoff)
        return None
    def cutoff(self):
        return self.avg_cutoff